Omar Mancilla Martínez

# Proyecto Final: 

## Plan de Trabajo
Plan de trabajo para el proyecto final de Tripleten.

## EDA
Iniciamos con la un análisis exploratorio de los datos para entender la información que tenemos y determinar el plan a seguir.

Objetivos del EDA: 
- Entender las cuatro fuentes de datos que tenemos.
- Revisar la calidad de las fuentes de datos
- Generar un diccionario de datos
- Determinar que pre-procesamiento es necesario
- Encontrar tendencias generales en los datos
- Determinar el desbalance de clases


### Carga de datos:
Cargaremos las librerias pertinentes y los datos en archivos CSV.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


In [5]:
df_personal = pd.read_csv("./datasets/personal.csv")
df_phone = pd.read_csv("./datasets/phone.csv ")
df_contract = pd.read_csv("./datasets/contract.csv")
df_internet = pd.read_csv("./datasets/internet.csv")

Ahora exploraremos los dataframes a traves de la siguiente función:

In [54]:
def explore(df): 
    display(df.head())
    print(df.info())
    print("\n shape:",df.shape)
    display(df.describe())
    print("Cantidad de datos nulos:\n",df.isnull().sum(),"\n")
    
    print("Valores comunes:")
    for columna in df.columns:
        print(f"Valores más comunes en la columna '{columna}':")
        print(df[columna].value_counts())
        
        
    

#### Personal:

In [55]:
explore(df_personal)

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
None

 shape: (7043, 5)


,SeniorCitizen
count,7043.000000
mean,0.162147
std,0.368612
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Cantidad de datos nulos:
 customerID       0
gender           0
SeniorCitizen    0
Partner          0
Dependents       0
dtype: int64 

Valores comunes:
Valores más comunes en la columna 'customerID':
customerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64
Valores más comunes en la columna 'gender':
gender
Male      3555
Female    3488
Name: count, dtype: int64
Valores más comunes en la columna 'SeniorCitizen':
SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64
Valores más comunes en la columna 'Partner':
Partner
No     3641
Yes    3402
Name: count, dtype: int64
Valores más comunes en la columna 'Dependents':
Dependents
No     4933
Yes    2110
Name: count, dtype: int64


##### Observaciones:
- Notamos que no tenemos valores nulos en el dataframe
- podemos codificar la mayoria de las columnas, pues son binarias. Como "gender", "partner", "seniorCitizen", 
- Solo el 16% son senior, segun SeniorCitizen

#### Contract:

In [56]:
explore(df_contract)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None

 shape: (7043, 8)


,MonthlyCharges
count,7043.000000
mean,64.761692
std,30.090047
min,18.250000
25%,35.500000
50%,70.350000
75%,89.850000
max,118.750000


Cantidad de datos nulos:
 customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64 

Valores comunes:
Valores más comunes en la columna 'customerID':
customerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64
Valores más comunes en la columna 'BeginDate':
BeginDate
2014-02-01    366
2019-10-01    237
2019-11-01    237
2019-09-01    237
2020-01-01    233
             ... 
2020-02-01     11
2014-01-01      7
2013-10-01      3
2013-12-01      3
2013-11-01      2
Name: count, Length: 77, dtype: int64
Valores más comunes en la columna 'EndDate':
EndDate
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name:

##### Observaciones:
- Las columnas de fechas tienen un tipo incorrecto.
- ```TotalCharges``` tiene el tipo incorrecto.
- ```Type,	PaperlessBilling,	PaymentMethod``` pueden ser codificados.
- ```MonthlyCharges``` parece estar distribuido de forma normal.
- ```EndDate``` usa "NO" para seleccionar clientes que no han terminado su contrato, evitando poder usar el formato correcto.

#### Internet:

In [57]:
explore(df_internet)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
None

 shape: (5517, 8)


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
count,5517,5517,5517,5517,5517,5517,5517,5517
unique,5517,2,2,2,2,2,2,2
top,7590-VHVEG,Fiber optic,No,No,No,No,No,No
freq,1,3096,3498,3088,3095,3473,2810,2785


Cantidad de datos nulos:
 customerID          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64 

Valores comunes:
Valores más comunes en la columna 'customerID':
customerID
7590-VHVEG    1
5181-OABFK    1
9356-AXGMP    1
5176-OLSKT    1
7508-DQAKK    1
             ..
1104-FEJAM    1
2972-YDYUW    1
4238-JSSWH    1
0869-PAPRP    1
3186-AJIEK    1
Name: count, Length: 5517, dtype: int64
Valores más comunes en la columna 'InternetService':
InternetService
Fiber optic    3096
DSL            2421
Name: count, dtype: int64
Valores más comunes en la columna 'OnlineSecurity':
OnlineSecurity
No     3498
Yes    2019
Name: count, dtype: int64
Valores más comunes en la columna 'OnlineBackup':
OnlineBackup
No     3088
Yes    2429
Name: count, dtype: int64
Valores más comunes en la columna 'DeviceProtection':
DeviceProtection
No     3095
Yes    2422
Name: count, dtype: int64
Valor

##### Observaciones:
- No tenemos datos nulos
- Todas las columnas excepto el id de cliente, pueden ser codificadas.

#### Phone:

In [58]:
explore(df_phone)

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
None

 shape: (6361, 2)


,customerID,MultipleLines
count,6361,6361
unique,6361,2
top,5575-GNVDE,No
freq,1,3390


Cantidad de datos nulos:
 customerID       0
MultipleLines    0
dtype: int64 

Valores comunes:
Valores más comunes en la columna 'customerID':
customerID
5575-GNVDE    1
0980-FEXWF    1
8082-GHXOP    1
3393-FMZPV    1
8701-DGLVH    1
             ..
8197-BFWVU    1
9154-QDGTH    1
8499-BRXTD    1
8552-OBVRU    1
3186-AJIEK    1
Name: count, Length: 6361, dtype: int64
Valores más comunes en la columna 'MultipleLines':
MultipleLines
No     3390
Yes    2971
Name: count, dtype: int64


##### Observaciones:
- No tenemos datos nulos
- La unica columna puede ser codificada.

### Objetivos:


### Dicciónario de datos:

### Carga de datos:


## Pre procesamiento:

### Datos nulos y duplicados:


### Separación de caracteristicas:



### Balanceo de clases:

### Escalado y codificación de caracteristicas:


### División del dataset:


## Analsisi de los datos:


### Generar modelos de Machine Learning:


#### Modelo Dummy:

#### Arbol de decisiones:


#### Random Forest:


#### Regresión Logística

#### Decenso de gradiente

### Evaluación de los modelos:


#### Curva ROC

#### Valor PR

#### Exploración de resultados:

### Exploración del mejor modelo:

#### Puntuajes

#### Curva ROC

#### Curva PR

## Conclusiones